In [ ]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借車量:int = Field(alias='available_rent_bikes') 
    可還車量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借車量','可還車量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df

In [ ]:
df1 = df.groupby(by='行政區域')[['總數量','可借車量','可還車量']].agg([('車輛數','sum'),('站點數','count')])
df1

In [ ]:
#pivot
#stack,unstack
#欄位有名稱
s1 = df1.stack(level=[0,1])
s1.index.names = ['行政區域','主題','數量']
s1.unstack(level=['數量'])